# get subset of data

In [2]:
import os
import random
from pathlib import Path

dir = Path('dataset/cifar10-small')

for subset in ['train', 'test']:
    for class_ in os.listdir(dir / subset):
        path= dir / subset / class_
        
        image_files = os.listdir(path)
        print(subset, '-', class_, '-', len(image_files))

train - deer - 1000
train - cat - 1000
train - horse - 1000
train - dog - 1000
train - automobile - 1000
train - bird - 1000
train - frog - 1000
train - truck - 1000
train - airplane - 1000
train - ship - 1000
test - deer - 200
test - cat - 200
test - horse - 200
test - dog - 200
test - automobile - 200
test - bird - 200
test - frog - 200
test - truck - 200
test - airplane - 200
test - ship - 200


In [3]:
dataset_size= 0.2
DELETE= False

if DELETE:
    for subset in ['train', 'test']:
        for class_ in os.listdir(dir / subset):
            path= dir / subset / class_
            
            image_files = os.listdir(path)

            # get random images based on intended dataset_size
            num_images_to_delete = int((1-dataset_size) * len(image_files))
            images_to_delete = random.sample(image_files, num_images_to_delete)

            # delete images
            for image in images_to_delete:
                image_path = os.path.join(path, image)
                os.remove(image_path)

In [4]:
dir = Path('dataset/cifar10-small')

for subset in ['train', 'test']:
    for class_ in os.listdir(dir / subset):
        path= dir / subset / class_
        
        image_files = os.listdir(path)
        print(subset, '-', class_, '-', len(image_files))

train - deer - 1000
train - cat - 1000
train - horse - 1000
train - dog - 1000
train - automobile - 1000
train - bird - 1000
train - frog - 1000
train - truck - 1000
train - airplane - 1000
train - ship - 1000
test - deer - 200
test - cat - 200
test - horse - 200
test - dog - 200
test - automobile - 200
test - bird - 200
test - frog - 200
test - truck - 200
test - airplane - 200
test - ship - 200


# get dataset as array

In [5]:
import os
import numpy as np
from PIL import Image

In [6]:
def load_image_dataset(root_dir, shuffle= True):
    
    class_labels = []
    images = []
    
    for class_dir in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_dir)
        if os.path.isdir(class_path):
            class_label = class_dir
            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)
                image = Image.open(image_path)
                image = np.array(image)
                class_labels.append(class_label)
                images.append(image)

    images = np.array(images)
    class_labels = np.array(class_labels)
    
    if shuffle:
        shuffle_indices = np.random.permutation(len(images))
        images = images[shuffle_indices]
        class_labels = class_labels[shuffle_indices]
    
    return images, class_labels

train = 'dataset/cifar10-small/train'
x_train, y_train= load_image_dataset(train)

test = 'dataset/cifar10-small/test'
x_test, y_test= load_image_dataset(test)

# get validation set
perc_vals= 0.2
n_vals= int(len(x_train) * perc_vals)

x_val= x_train[:n_vals]
y_val= y_train[:n_vals]

x_train= x_train[n_vals:]
y_train= y_train[n_vals:]

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(8000, 32, 32, 3) (8000,)
(2000, 32, 32, 3) (2000,)
(2000, 32, 32, 3) (2000,)


# with TF

In [7]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import image_dataset_from_directory
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dense, Activation, Flatten, Dropout

2023-07-16 08:17:43.926414: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# normalize the data
mean= np.mean(x_train)
std= np.std(x_train)

x_train_clean= (x_train - mean)/std
x_val_clean= (x_val - mean)/std
x_test_clean= (x_test - mean)/std

# convert labels to one-hot
encoder= OneHotEncoder(sparse_output= False, drop= None, handle_unknown= 'ignore')
encoder.fit(y_train.reshape(-1, 1))
y_train_clean= encoder.transform(y_train.reshape(-1, 1))
y_val_clean= encoder.transform(y_val.reshape(-1, 1))
y_test_clean= encoder.transform(y_test.reshape(-1, 1))

In [9]:
datagen= ImageDataGenerator(
    rotation_range= 15, 
    width_shift_range= 0.1, 
    height_shift_range= 0.1,
    horizontal_flip= True,
    vertical_flip= True
)
datagen.fit(x_train_clean)
datagen

In [10]:
x_train_clean.shape[1:]

(32, 32, 3)

In [11]:
y_train_clean.shape[-1]

10

In [12]:
base_hidden_unit= 32
num_classes= y_train_clean.shape[-1]
weight_decay= 1e-4
model= keras.Sequential()

# conv 1
model.add(Conv2D(filters= base_hidden_unit, 
                 kernel_size= (3, 3),
                 padding= 'same', 
                 kernel_regularizer= keras.regularizers.l2(weight_decay),
                 input_shape= x_train_clean.shape[1:]
                 ))
model.add(Activation('relu'))
model.add(BatchNormalization())

# conv 2
model.add(Conv2D(filters= base_hidden_unit, 
                 kernel_size= (3, 3),
                 padding= 'same', 
                 kernel_regularizer= keras.regularizers.l2(weight_decay),
                 ))
model.add(Activation('relu'))
model.add(BatchNormalization())

# pool + dropout
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.2))

# conv 3
model.add(Conv2D(filters= base_hidden_unit * 2, 
                 kernel_size= (3, 3),
                 padding= 'same', 
                 kernel_regularizer= keras.regularizers.l2(weight_decay),
                 ))
model.add(Activation('relu'))
model.add(BatchNormalization())

# conv 4
model.add(Conv2D(filters= base_hidden_unit *2, 
                 kernel_size= (3, 3),
                 padding= 'same', 
                 kernel_regularizer= keras.regularizers.l2(weight_decay),
                 ))
model.add(Activation('relu'))
model.add(BatchNormalization())

# pool + dropout
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.3))

# conv 5
model.add(Conv2D(filters= base_hidden_unit * 4, 
                 kernel_size= (3, 3),
                 padding= 'same', 
                 kernel_regularizer= keras.regularizers.l2(weight_decay),
                 ))
model.add(Activation('relu'))
model.add(BatchNormalization())

# conv 6
model.add(Conv2D(filters= base_hidden_unit * 4, 
                 kernel_size= (3, 3),
                 padding= 'same', 
                 kernel_regularizer= keras.regularizers.l2(weight_decay),
                 ))
model.add(Activation('relu'))
model.add(BatchNormalization())

# pool + dropout
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.4))

# fc
model.add(Flatten())
model.add(Dense(num_classes, activation= 'softmax'))

model.summary()

2023-07-16 08:17:46.530468: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 08:17:46.537650: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                        

In [13]:
TRAIN= False
batch_size= 16
epochs= 100
model_path= 'output/model.hdf5'

checkpoint= ModelCheckpoint(filepath= model_path, save_best_only= True, verbose= 1)
optimizer= keras.optimizers.Adam(lr=0.0001,
                                 weight_decay= 1e-6)

model.compile(loss='categorical_crossentropy', 
              optimizer= optimizer,
              metrics= ['accuracy'])

if TRAIN:
    history= model.fit_generator(
        datagen.flow(x_train_clean, y_train_clean, batch_size= batch_size),
        callbacks= checkpoint, 
        steps_per_epoch= x_train_clean.shape[0] // batch_size, 
        epochs= epochs,
        verbose= 1, 
        validation_data= (x_val_clean, y_val_clean)
        )

In [16]:
saved_model= keras.models.load_model(model_path)
saved_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                      

In [17]:
scores = saved_model.evaluate(x_test_clean, 
                              y_test_clean, 
                              batch_size=128, 
                              verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

16/16 [==============================] - 5s 252ms/step - loss: 1.0534 - accuracy: 0.6870

Test result: 68.700 loss: 1.053


# with pytorch

In [141]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

import tqdm

In [21]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(8000, 32, 32, 3) (8000,)
(2000, 32, 32, 3) (2000,)
(2000, 32, 32, 3) (2000,)


In [25]:
x_train.shape

(8000, 32, 32, 3)

In [27]:
#normalize 

mean= np.mean(x_train)
std= np.std(x_train)

x_train_clean= (x_train-mean)/std
x_val_clean= (x_val-mean)/std
x_test_clean= (x_test-mean)/std

In [49]:
x_train_clean.reshape(-1, 3, 32, 32).shape

(8000, 3, 32, 32)

In [124]:
# cast to tensor
x_train_tensor= torch.Tensor(x_train_clean.reshape(-1, 3, 32, 32))
x_val_tensor= torch.Tensor(x_val_clean.reshape(-1, 3, 32, 32))
x_test_tensor= torch.Tensor(x_test_clean.reshape(-1, 3, 32, 32))

y_train_tensor= torch.Tensor(np.argmax(y_train_clean, axis= 1)).long()
y_val_tensor= torch.Tensor(np.argmax(y_val_clean, axis= 1)).long()
y_test_tensor= torch.Tensor(np.argmax(y_test_clean, axis= 1)).long()

In [125]:
x_train_tensor.shape

torch.Size([8000, 3, 32, 32])

In [126]:
# create dataloader
train_ds= TensorDataset(x_train_tensor, y_train_tensor)
val_ds= TensorDataset(x_val_tensor, y_val_tensor)

batch_size= 8
train_dl= DataLoader(train_ds, batch_size= batch_size)
val_dl= DataLoader(val_ds, batch_size= batch_size)

In [135]:
# create model 

class MyCNN(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1= nn.Conv2d(in_channels= 3, 
                              out_channels= 32, 
                              kernel_size= (3, 3),
                              padding= 'same',
                              )
        self.conv2= nn.Conv2d(in_channels= 32, 
                              out_channels= 32, 
                              kernel_size= (3, 3),
                              padding= 'same',
                              )
        self.conv3= nn.Conv2d(in_channels= 32, 
                              out_channels= 64, 
                              kernel_size= (3, 3),
                              padding= 'same',
                              )
        self.conv4= nn.Conv2d(in_channels= 64, 
                              out_channels= 64, 
                              kernel_size= (3, 3),
                              padding= 'same',
                              )
        self.conv5= nn.Conv2d(in_channels= 64, 
                              out_channels= 128, 
                              kernel_size= (3, 3),
                              padding= 'same',
                              )
        self.conv6= nn.Conv2d(in_channels= 128, 
                              out_channels= 128, 
                              kernel_size= (3, 3),
                              padding= 'same',
                              )
        self.pool= nn.MaxPool2d(kernel_size= (2, 2))
        self.batch_norm32= nn.BatchNorm2d(32)
        self.batch_norm64= nn.BatchNorm2d(64)
        self.batch_norm128= nn.BatchNorm2d(128)
        self.dense= nn.Linear(in_features= 2048, 
                              out_features= 10)

    def forward(self, x):
        # conv 1
        x= self.batch_norm32(F.relu(self.conv1(x)))
        # conv 2
        x= self.batch_norm32(F.relu(self.conv2(x)))
        
        # pool & dropout
        x= F.dropout2d(self.pool(x), p= 0.2)
        
        # conv 3
        x= self.batch_norm64(F.relu(self.conv3(x)))
        # conv 4
        x= self.batch_norm64(F.relu(self.conv4(x)))
        
        # pool & dropout
        x= F.dropout2d(self.pool(x), p= 0.3)
        
        # conv 5
        x= self.batch_norm128(F.relu(self.conv5(x)))
        # conv 6
        x= self.batch_norm128(F.relu(self.conv6(x)))

        # pool & dropout
        x= F.dropout2d(self.pool(x), p= 0.4)

        # fc
        x= nn.Flatten()(x)
        x= F.softmax(self.dense(x), dim= 0)

        return x
    
model= MyCNN()
sample= torch.ones(8, 3, 32, 32)
model(sample).shape

torch.Size([8, 10])

In [149]:
my_cnn= MyCNN()
loss_fn= nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(my_cnn.parameters(), lr=0.0001)

n_epochs= 100
log_print= 1

n_train_samples= len(train_ds)
n_val_samples= len(val_ds)

In [150]:
acc_hist_train= [0] * n_epochs
acc_hist_val= [0] * n_epochs

loss_hist_train= [0] * n_epochs
loss_hist_val= [0] * n_epochs

for epoch in range(n_epochs):
    
    # training
    my_cnn.train()
    for X_batch, y_batch in train_dl:
        
        y_pred_proba= my_cnn(X_batch)
        y_pred= torch.argmax(y_pred_proba, dim= 1)

        loss= loss_fn(y_pred_proba, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        is_correct= (y_pred == y_batch).float()
        n_correct= is_correct.sum()

        acc_hist_train[epoch] += n_correct.item()
        loss_hist_train[epoch] += loss.detach().item()
    
    acc_hist_train[epoch] /= n_train_samples
    loss_hist_train[epoch] /= n_train_samples

    # validation
    my_cnn.eval()
    with torch.no_grad():
        for X_batch, y_batch in val_dl:
            
            y_pred_proba= my_cnn(X_batch)
            y_pred= torch.argmax(y_pred_proba, dim= 1)

            loss= loss_fn(y_pred_proba, y_batch)

            is_correct= (y_pred == y_batch).float()
            n_correct= is_correct.sum()

            acc_hist_val[epoch] += n_correct.item()
            loss_hist_val[epoch] += loss.detach().item()
        
        acc_hist_val[epoch] /= n_val_samples
        loss_hist_val[epoch] /= n_val_samples

    if epoch % log_print == 0:
        acc= acc_hist_train[epoch]
        loss= loss_hist_train[epoch]
        val_acc= acc_hist_val[epoch]
        val_loss= loss_hist_val[epoch]

        print(f'epoch {epoch + 1} : acc = {acc: .4f} - val acc = {val_acc: .4f} - '
        f'loss = {loss: .4f} - val loss = {val_loss: .4f}'
        )

epoch 1 : acc =  0.2167 - val acc =  0.2285 - loss =  0.2759 - val loss =  0.2738
epoch 2 : acc =  0.2506 - val acc =  0.2400 - loss =  0.2710 - val loss =  0.2728
epoch 3 : acc =  0.2707 - val acc =  0.2725 - loss =  0.2680 - val loss =  0.2696
epoch 4 : acc =  0.2893 - val acc =  0.2680 - loss =  0.2667 - val loss =  0.2704
epoch 5 : acc =  0.2960 - val acc =  0.2650 - loss =  0.2658 - val loss =  0.2698
epoch 6 : acc =  0.3041 - val acc =  0.2835 - loss =  0.2643 - val loss =  0.2682
epoch 7 : acc =  0.3021 - val acc =  0.2645 - loss =  0.2638 - val loss =  0.2697
epoch 8 : acc =  0.3245 - val acc =  0.2620 - loss =  0.2619 - val loss =  0.2709
epoch 9 : acc =  0.3247 - val acc =  0.2565 - loss =  0.2616 - val loss =  0.2707
epoch 10 : acc =  0.3355 - val acc =  0.2665 - loss =  0.2606 - val loss =  0.2688
epoch 11 : acc =  0.3371 - val acc =  0.2615 - loss =  0.2603 - val loss =  0.2693
epoch 12 : acc =  0.3510 - val acc =  0.2920 - loss =  0.2589 - val loss =  0.2681
epoch 13 : ac